2023-10-17

What follows is a quick tutorial on using [patchworklib](https://pypi.org/project/patchworklib/) v0.6.2 to combine a seaborn Axes level plot and a Figure level plot for simultaneous display. The Motivation stems from the desire to construct 'dashboards' of plots on the fly, for example density plots alongside correlation, or relational plots alongside violin plots, as displayed below. Unfortunately, seaborn does not support natively combining Figure level plots with other figures through the matplotlib subfigure API at this time, and other methods that i have experimented with thus far have proved insufficient.

Note: patchworklib currently does not provide acceptable documentation or error handling so getting this far was a struggle.

In [ ]:
# set up environment

import seaborn as sns
import patchworklib as pw

# for handling figure level plots, patchworklib overwrites (overloads?) seaborn classes. This line is necessary for later behavior
pw.overwrite_axisgrid()

# an example dataset
penguins = sns.load_dataset("penguins")
penguins.head()

First we'll construct a figure level plot, for example a relational plot between 6 variables in the penguins dataset. The Facetgrid object returned is then 'loaded' into a `pw.Brick` object for display handling.

In [ ]:
# construct a figure level plot and assign it to a variable

rp = sns.relplot(penguins, x='bill_length_mm', y='bill_depth_mm', size ='flipper_length_mm', row='sex',col='species', hue='body_mass_g'
                 )
display(rp.tight_layout())

# Figure level plots are 'loaded' into pw.Brick objects which return a Brick object

pwg = pw.load_seaborngrid(rp)

Differing from the Figure level plots, in the Axes level plots we first define a `pw.Brick` object which can be used as an Axes in the same was as `plt.subplots` Axes objects.

In [ ]:

# Different behavior to the figure level plot, Axes level plots need to be assigned to
# predefined axes returned by the instantiation of a `pw.Brick` object

# instantiate `pw.Brick`
vp = pw.Brick(figsize=(3,2))
print(type(vp))
# Construct an Axes level plot and assign it to a Brick
p = sns.violinplot(penguins, x = 'species',y='body_mass_g', ax=vp)

p

Now that the Figure level and Axes level plots are handled by `pw.Brick` objects, they can be treated the same way. Below we horizontally stack and vertically stack the predefined plots:

In [ ]:
# display the composite figure using the pipe operator resulting in a horizontal stack
vp | pwg

In [ ]:
# display the composite figure using the slash operator resulting in a vertical stack

vp / pwg

Overall I am happy with the design of this library, but as I said earlier, it is not user-friendly. The documentation needs serious work, and what is currently stopping me from using it is that the display appears to be using its own default themes including transparent backgrounds, making them unusable in dark themes. Hopefully the dev adds documentation or clearer setting handling in the future.